# Calculating extinction and reddening values

Written by Evgenii N.


We want to fit Girardi isochrones for different combinations of filters: B-I, V-I, R-I etc. In order to do that we need to know the extinction (e.g. Av) and reddening numbers (e.g. E(B-V)).

In [1]:
# Import the libraries
import os
import pandas as pd
from tabulate import tabulate

In [2]:
def calculate_extinction(filter_name, relative_extinctions, a_v):
    """
    Calculate light extinction for each filter.
    
    Parameters
    -----------
    
    filter_name: float
        Name of a filter, i.e. "B", "I" etc.
        
    relative_extinctions: dict
       key: str
           Name of filter
       value: float
           Value A_lambda / A_V, the ratio of the current filter's extinction to
           the extinction for the V-band `a_v`.
           
    a_v: float
        The light extinction for the visual band.
    
    Returns
    ---------
    
    Light extinction value for the given filter.
    """

    return relative_extinctions[filter_name] * a_v

def save_to_csv(data_dir, extinctions):
    """
    Save extinctions to a csv file.
    
    Parameters
    ----------
    
    data_dir: str
        Name of the directory for the data file.
        
    extinctions: dict
        key: str
            Filter name, i.e. "B", "I" etc.
        value: float
            Light extinction value for the filter.
            
    Return
    ------
    
    pandas.core.frame.DataFrame
        Table containing the extinctions.
    """

    s = pd.Series(extinctions, name='Extinction')
    s.index.name = 'Filter'

    if not os.path.isdir(data_dir):
        os.mkdir(data_dir)

    path_to_csv = os.path.join(data_dir, "extinctions.csv")
    s.to_csv(path_to_csv)
    return pd.DataFrame(s)
    
def calculate_extinctions(relative_extinctions):
    """
    Calculate extinctions for all filters.
    
    Parameters
    ----------
    
    relative_extinctions: dict
       key: str
           Name of filter
       value: float
           Value A_lambda / A_V, the ratio of the current filter's extinction to
           the extinction for the V-band `a_v`.
           
    Returns
    --------
    
    Dictionary containing extinction values for all filters.
    
    dict
        key: str
            Filter name, i.e. "B", "I" etc.
        value: float
            Light extinction value for the filter.
    
    """
    
    # Get names of filters into a list
    filter_names = relative_extinctions.keys()

    return {
        filter_name: calculate_extinction(
            filter_name=filter_name, relative_extinctions=relative_extinctions, a_v=a_v)
        for filter_name in filter_names
    }

In [3]:
# Reddening E(B-V)
# Alcaino 1976, https://ui.adsabs.harvard.edu/abs/1976A%26AS...26..251A/abstract
e_b_minus_v = 0.14

# Rotal to selective extinction ratio R(V)
# Cardelli et al. (1989) https://ui.adsabs.harvard.edu/abs/1989ApJ...345..245C/abstract
r_v = 3.1

# Light extinction extinction A(V)
a_v = r_v * e_b_minus_v

# Values A_lambda / A_V form Table A4 in Gordon et al. (2003)
# https://ui.adsabs.harvard.edu/abs/2003ApJ...594..279G/abstract
#
# For example, A_B / A_V = 1.37, A_R / A_V = 0.8 etc.
relative_extinctions = dict(
    B=1.37,
    V=1.0,
    R=0.8,
    I=0.57
)

# Calcualte the extinctions
extinctions = calculate_extinctions(relative_extinctions=relative_extinctions)

# Save to a CSV file for later use
df = save_to_csv(data_dir="data", extinctions=extinctions)

# Print the values here in a table
print("\nTable 1: Extinction values for bands.\n")
headers = ['Filter', 'Extinction [mag]']
print(tabulate(df, headers=headers, floatfmt=".2f", tablefmt="github"))


Table 1: Extinction values for bands.

| Filter   |   Extinction [mag] |
|----------|--------------------|
| B        |               0.59 |
| V        |               0.43 |
| R        |               0.35 |
| I        |               0.25 |


## Are our extinction values any good?


Extinction values are shown in Table 1. We can see that bluer filter have higher extinction, which makes sense, because shorter wavelengths are scattered more. Also, the V-band extinction is equation 0.43, as expected, since 

R(V) E(B-V) = 3.1 * 0.14 = 0.43

I'm happy with the results.

## How do we calculate reddening values?

Now that I got all the extinction values, I can calculate the reddening value E(X-Y) for any combination of filters X,Y using the definition of reddening:

$$
E(X-Y) = A_X - A_Y,
$$

where $A_X$ and $A_Y$ are light extinction values for any two filters from Table 1.